In [12]:
import time
import torch
import triton
import triton.language as tl
import torch.nn.functional as F

In [11]:
sample=torch.tensor(
    [
        [1,2,3],
        [5,6,7]
    ],
    dtype=torch.float32,
    device='cuda'
)

out=F.softmax(
    sample,
    dim=1
)
out

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]], device='cuda:0')

In [26]:
def softmax(
        x:torch.Tensor
)-> torch.Tensor:
    x_max=x.max(dim=1)[0]
    safe_x=x-x_max[:,None]

    output=torch.exp(
        safe_x
    )
    output=output/output.sum(dim=1)[:,None]
    
    return output

In [59]:
a=time.time()
softmax(sample)
time.time()-a

0.0009968280792236328

In [ ]:
@triton.jit

def forward():
    pass

def softmax_triton(x:torch.Tensor)-> torch.Tensor:
    rows,columns=x.shape
     
